In [1]:
import ee
import geemap
import folium
import time

ee.Initialize()
Map = geemap.Map()

# Define the region of interest of Amazon basin
amazon_region = ee.Geometry.Polygon([[[-80.0, 10.0],
            [-20.0, 10.0],
            [-20.0, -50.0],                           
            [-80.0, -50.0],
            [-80.0, 10.0]]]) 

print("START")

# Define the bitmasks
cloud_bit_mask = ee.Number(1 << 5)  # Cloud bit is in the 6th bit position
cirrus_bit_mask = ee.Number(1 << 9)  # Cirrus bit is in the 10th bit position

# Apply the mask using bitwise AND to check that both cloud and cirrus bits are 0
def mask_clouds(image):
    qa = image.select('QA60')  # Select the QA60 band that holds cloud and cirrus bit information
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask)



sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(amazon_region) \
                .filterDate('2022-01-01', '2022-12-31') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

sentinel2 = sentinel2.map(mask_clouds)

# Calculate NDVI
ndvi = sentinel2.map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('NDVI')).median()
# Calculate EVI
def calculate_evi(image):
    return image.expression(
        '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
        {
            'B8': image.select('B8'),
            'B4': image.select('B4'),
            'B2': image.select('B2')
        }).rename('EVI')

evi = sentinel2.map(calculate_evi).median()


# Define visualization parameters
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2']
}
# display sentinel-2 data
Map.addLayer(sentinel2, visualization,'sentinel2');
Map.centerObject(amazon_region, 6)
Map

START


Map(center=[-19.307052700621895, -49.99999999999999], controls=(WidgetControl(options=['position', 'transparen…